# Gaussian process with vector inputs

In this notebook we look into Gaussian process regression for function of the form $f : \mathbb{R}^n \rightarrow \mathbb{R}$. 

We will assume an offline scenario where the training dataset $\mathcal{D} = (\mathbf{x}_i, y_i)_{i=1,...,N}$ is already available with $\mathbf{x}_i \in \mathbb{R}^3, y_i \in \mathbb{R}$.

In [1]:
import os

os.getcwd()

'/home/matthijs/bsc/BachelorProject/notebooks'

In [2]:
import torch
# Define the training data
train_x = torch.randn(100, 3)  # 100 data points, each with 3-dimensional input
train_y = torch.sin(train_x[:, 0])  # Target function (sine of first dimension)

In [6]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(torch.zeros(1, 3), torch.zeros(1), likelihood)

In [7]:
from gpytorch import ExactMarginalLogLikelihood

# Initialize likelihood and model
model.train()
likelihood.train()

model.set_train_data(train_x, train_y, strict=False)

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = ExactMarginalLogLikelihood(likelihood, model)

training_iter=50

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Iter 1/50 - Loss: 1.050   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 1.001   lengthscale: 0.744   noise: 0.644
Iter 3/50 - Loss: 0.953   lengthscale: 0.798   noise: 0.598
Iter 4/50 - Loss: 0.904   lengthscale: 0.854   noise: 0.554
Iter 5/50 - Loss: 0.855   lengthscale: 0.913   noise: 0.513
Iter 6/50 - Loss: 0.806   lengthscale: 0.974   noise: 0.474
Iter 7/50 - Loss: 0.758   lengthscale: 1.036   noise: 0.437
Iter 8/50 - Loss: 0.710   lengthscale: 1.101   noise: 0.402
Iter 9/50 - Loss: 0.662   lengthscale: 1.167   noise: 0.369
Iter 10/50 - Loss: 0.615   lengthscale: 1.235   noise: 0.339
Iter 11/50 - Loss: 0.568   lengthscale: 1.303   noise: 0.310
Iter 12/50 - Loss: 0.523   lengthscale: 1.372   noise: 0.284
Iter 13/50 - Loss: 0.478   lengthscale: 1.442   noise: 0.259
Iter 14/50 - Loss: 0.434   lengthscale: 1.511   noise: 0.237
Iter 15/50 - Loss: 0.391   lengthscale: 1.579   noise: 0.216
Iter 16/50 - Loss: 0.349   lengthscale: 1.646   noise: 0.196
Iter 17/50 - Loss: 0.307   length

In [8]:
# Set the model and likelihood into eval mode
model.eval()
likelihood.eval()

# Make predictions
test_x = torch.randn(10, 3)  # 10 test points
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))

print("Predicted means:", observed_pred.mean)
print("Predicted variances:", observed_pred.variance)

Predicted means: tensor([ 0.9575,  0.0971, -0.9676, -0.0219,  0.5669,  0.4681, -0.5755,  0.8963,
        -0.4611, -1.0742])
Predicted variances: tensor([0.0080, 0.0074, 0.0066, 0.0066, 0.0070, 0.0357, 0.0063, 0.0067, 0.0062,
        0.0190])
